Importation des library

In [1]:
import numpy as np
import pandas as pd
import vectorbt as vbt

# ============================================================
# CONFIGURATION DATASETS
# ============================================================

DATASETS = {
    "BTCUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-25_4h_data",
    "ETHUSDT_4h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\4h\ETHUSDT\ETHUSDT_1years_2024-12-22_TO_2025-12-26_4h_data",
    "BTCUSDT_1h": r"C:\Users\gunsa\Desktop\Git-Repo\Bot-Trading\Data\1h\BTCUSDT\BTCUSDT_1years_2024-12-22_TO_2025-12-26_1h_data",
}

# ============================================================
# PARAMÈTRES STRATÉGIE (communs à tous les datasets)
# ============================================================

RSI_LENGTH = 14
LOOKBACK_LEFT = 3
LOOKBACK_RIGHT = 3
MIN_DIV_BARS = 10
MAX_DIV_BARS = 200

STOP_LOSS_PCT = 0.03
TAKE_PROFIT_PCT = 0.06

FEES = 0.001
INIT_CASH = 1000

# ============================================================
# STRATÉGIE RSI DIVERGENCES (LONG ONLY)
# ============================================================

def run_rsi_divergence_strategy(df: pd.DataFrame) -> vbt.Portfolio:
    # -------------------------
    # Sécurisation des données
    # -------------------------
    high = df['High'].astype(np.float64)
    low = df['Low'].astype(np.float64)
    close = df['Close'].astype(np.float64).ffill()

    # -------------------------
    # RSI
    # -------------------------
    rsi = vbt.RSI.run(close, window=RSI_LENGTH).rsi

    # -------------------------
    # Pivots
    # -------------------------
    window = LOOKBACK_LEFT + LOOKBACK_RIGHT + 1

    pivot_low_price = (
        low.shift(LOOKBACK_RIGHT) ==
        low.rolling(window).min()
    )

    pivot_low_rsi = (
        rsi.shift(LOOKBACK_RIGHT) ==
        rsi.rolling(window).min()
    )

    # -------------------------
    # Divergences
    # -------------------------
    bullish_div = pd.Series(False, index=df.index)
    hidden_bullish_div = pd.Series(False, index=df.index)

    idx = np.where(pivot_low_price & pivot_low_rsi)[0]

    if len(idx) >= 2:
        idx1 = idx[1:]
        idx2 = idx[:-1]
        bars_between = np.abs(idx1 - idx2)

        # Divergence haussière régulière
        valid_regular = (
            (low.iloc[idx1].values < low.iloc[idx2].values) &
            (rsi.iloc[idx1].values > rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        # Divergence haussière cachée
        valid_hidden = (
            (low.iloc[idx1].values > low.iloc[idx2].values) &
            (rsi.iloc[idx1].values < rsi.iloc[idx2].values) &
            (bars_between >= MIN_DIV_BARS) &
            (bars_between <= MAX_DIV_BARS)
        )

        # Repositionnement au bar du pivot (TradingView-like)
        sig_regular = idx1[valid_regular] - LOOKBACK_RIGHT
        sig_hidden = idx1[valid_hidden] - LOOKBACK_RIGHT

        bullish_div.iloc[sig_regular[sig_regular >= 0]] = True
        hidden_bullish_div.iloc[sig_hidden[sig_hidden >= 0]] = True

    # -------------------------
    # Entrées LONG
    # -------------------------
    entries = bullish_div | hidden_bullish_div
    entries = entries.shift(1).fillna(False).astype(bool)

    # -------------------------
    # Backtest vectorbt
    # -------------------------
    portfolio = vbt.Portfolio.from_signals(
        close=close,
        entries=entries,
        sl_stop=STOP_LOSS_PCT,
        tp_stop=TAKE_PROFIT_PCT,
        init_cash=INIT_CASH,
        fees=FEES,
        direction='longonly'
    )

    return portfolio

# ============================================================
# RUN MULTI-DATASETS
# ============================================================

def run_all_datasets():
    results = []

    for name, path in DATASETS.items():
        print(f"▶ Backtest : {name}")

        df = pd.read_parquet(path)
        pf = run_rsi_divergence_strategy(df)

        stats = pf.stats()
        stats['dataset'] = name
        stats['trades'] = pf.trades.count()

        results.append(stats)

    results_df = pd.DataFrame(results).set_index('dataset')
    return results_df

# ============================================================
# MAIN
# ============================================================

if __name__ == "__main__":
    results = run_all_datasets()
    print("\n===== RÉSULTATS MULTI-DATASETS =====\n")
    print(results)


▶ Backtest : BTCUSDT_4h


C:\Users\gunsa\AppData\Local\Temp\ipykernel_15916\808285294.py:103: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



▶ Backtest : ETHUSDT_4h
▶ Backtest : BTCUSDT_1h


C:\Users\gunsa\AppData\Local\Temp\ipykernel_15916\808285294.py:103: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\gunsa\AppData\Local\Temp\ipykernel_15916\808285294.py:103: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




===== RÉSULTATS MULTI-DATASETS =====

                               Start                       End  \
dataset                                                          
BTCUSDT_4h 2024-12-22 01:00:00+01:00 2025-12-25 21:00:00+01:00   
ETHUSDT_4h 2024-12-22 01:00:00+01:00 2025-12-26 13:00:00+01:00   
BTCUSDT_1h 2024-12-22 00:00:00+01:00 2025-12-26 16:00:00+01:00   

                      Period  Start Value    End Value  Total Return [%]  \
dataset                                                                    
BTCUSDT_4h 369 days 00:00:00       1000.0  1164.813492         16.481349   
ETHUSDT_4h 369 days 16:00:00       1000.0  1278.200697         27.820070   
BTCUSDT_1h 369 days 17:00:00       1000.0  1752.607124         75.260712   

            Benchmark Return [%]  Max Gross Exposure [%]  Total Fees Paid  \
dataset                                                                     
BTCUSDT_4h             -9.159686                   100.0        32.738559   
ETHUSDT_4h        